# Excerise 1: Extracting station information from downloaded station netcdf files 
#### Meta data Temperature - gridded daily mean temperature in the Netherlands
Gridded files of daily mean temperature in the Netherlands. Based on 33 -35 automatic weather stations of the KNMI (https://dataplatform.knmi.nl/dataset/tg1-5).The interpolation method is Inverse Distance Weighted interpolation (IDW) using power parameter 2.0. Block size is 20km and search radius 110km. Due to use of a block it is not an exact interpolator: the interpolated value at a point can differ from the measured value. The number of observations changes from 15 (1961) to 26 (1991), 37 (1994) and 34 (2012). Version 5 has an updated extent.

More information avaiable from 	http://www.knmi.nl/bibliotheek/stageverslagen/stageverslag_Salet.pdf

In [ ]:
# Load required libraries
import os
import xarray as xr
import pandas as pd
import warnings
import glob
from scipy.interpolate import griddata
#import rasterio
#from rasterio.features import geometry_mask
import numpy as np
warnings.filterwarnings("ignore")
#from rasterio.transform import from_origin
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import plotly.express as px
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pickle
from scipy import stats
import random
import pickletools

##### Provide your inputs for data processing 

In [ ]:
data_path = "C:\\Users\\arumu002\\OneDrive - Wageningen University & Research\\Project I\\Rutger"
start_year = 1991 # choose the starting of climatological year
end_year = 2021 # choose the end of climatological year
station_list = data_path+"\\KNMI_weather_stations.csv" # Writing station after extracting from the netcdf files
station_list_pkl = data_path+"\\df_stations.pkl" # Writing station as pickle file after extracting from the netcdf files
ext_climate = data_path+"\\KNMI_weather_stations_data.csv" # Writing extracted weather information as csv
ext_climate_pkl = data_path+"\\KNMI_weather_stations_data.pkl" # Writing extracted weather information as pkl
shp1 = data_path+'\\shp\\NET_admin_0.shp' # Netherlands country level shapefile
shp2 = data_path+'\\shp\\NET_admin_1.shp' # Netherlands state level shapefile
shp3 = data_path+'\\shp\\agriculture_areas2.shp'# Nethelrands land based region shapefile
shp4 = data_path+'\\shp\\+station_points.shp' # write the station as shapfile

#### PART I - Extract list of stations avaiable from gridded data from 1990 to 2021
##### The list of stations over the years keep varying. So, this script extract the common stations based on coordinates over the current climatology period (1990-2022)

In [ ]:
if os.path.exists(station_list):
    print("List of common stations avaiable from the years 1990 to 2022 has been extracted. Please proceed to further steps.")
    df_stations = pd.read_csv(station_list)
    print("Overall, "+ str(len(df_stations))+" stations avaiable from 1990-2021")
else:
    print("Stations will be extracted from the years 1990-2022. Please wait until the process done")
    dataframes_list = []
    for i in range(start_year,end_year):
        file_paths = glob.glob(data_path + "\\" + str(i) + '\\*.nc')
        print("Merging daily files for the year: "+ str(i))
        ds = xr.open_mfdataset(file_paths, combine='by_coords')
        stations = ds['stations'].to_dataframe()
        stations1 = stations.dropna()
        list_stations = stations1.drop_duplicates() 
        list_stations.reset_index(drop=True, inplace=True)
        list_stations_f = list_stations.drop('stations', axis=1)
        dataframes_list.append(list_stations_f)
        print("Stations are getting extracted for the year: "+str(i)+" and " +str(len(list_stations_f))+ " stations found.")
# Initialize common_df with the first non-empty DataFrame
    df_stations = None
# Loop through the DataFrames
    for df in dataframes_list:
        if df_stations is None:
            df_stations = df
        elif not df.empty:
        # Use 'lon' and 'lat' columns for merging
            common_df = pd.merge(df_stations, df, on=['lon', 'lat'], how='inner')
            df_stations = common_df.drop_duplicates()
            df_stations.reset_index(drop=True, inplace=True)
            df_stations['ID'] = df_stations.index.map(lambda x: 'ID_{:03}'.format(x + 1))
# Display the common DataFrame
    if df_stations is not None:
        print("Overall, over the "+str(len(range(start_year,end_year)))+" years "+ str(len(df_stations)) +" stations found in common")
    else:
        print("No common DataFrame found.")
    df_stations.to_csv(station_list,index=False)
 # extract stations from all years and add into the list
filename = station_list_pkl
# Open the file in binary write mode and save the list of DataFrames using pickle
with open(filename, 'wb') as file:
    pickle.dump(df_stations, file)

#### PART II - Extracting climate information for avaiable stations from gridded data from 1990 to 2021
##### This script extract climate information for the years given and write as a database (.csv)

In [ ]:
if os.path.exists(ext_climate):
    print("Extracted climate information for the stataions from 1990 to 2021. Please proceed to next step ...")
else:
    print("Extracting climate information for the extraction loactaions from 1990 to 2021 has started. Please wait...")
    with open(station_list_pkl, 'rb') as file:
        df_stations = pickle.load(file)
    df_all = []
    for i in range(start_year,end_year):
        file_paths = glob.glob(data_path + "\\" + str(i) + '\\*.nc')
        print("Merging daily files for the year: " + str(i))
        ds = xr.open_mfdataset(file_paths, combine='by_coords')
        #################################################################################################################
        ######################### Extract rainfall for extracted staions netcdf file #######################
        stations_vals = ds['stationvalues'].to_dataframe()
        stations_vals1 = stations_vals.reset_index(level='time')
        stations_vals1.reset_index(drop=True, inplace=True)
        #station_filtered = stations_vals1[stations_vals1[['lon', 'lat']].isin(df_stations.to_dict(orient='list')).all(axis=1)]
        result_df = pd.merge(stations_vals1, df_stations, on=('lon', 'lat'), how='inner')
        #result_df1 = result_df.dropna(subset=['ID'])
        result_df['stationvalues'].fillna(0, inplace=True)
        print(result_df)
        df_wide_pivot = result_df.pivot(index='time', columns='ID', values='stationvalues')
        #print(df_wide_pivot)
        df_all.append(df_wide_pivot)
        #df_wide_pivot.to_csv(str(i) + ".csv")  # This part writes the raw climate files  
    df_f = pd.concat(df_all, ignore_index=False)
    #print(df_f)
    df_f.to_csv(ext_climate) # This part writes the raw climate files
    filename = ext_climate_pkl
# Open the file in binary write mode and save the list of DataFrames using pickle
    with open(filename, 'wb') as file:
        pickle.dump(df_f, file)

#### PART III: Plotting the stations
##### As the station IDs and names also varys over the years, this script creates a station ID with suffix of 'ID_' (i.e., ID_001, ID 002, etc). Which helps to identify station which need to be anlalyzed. 

In [ ]:
############ PLOT THE STATIONS #################
# Create a GeoDataFrame with 'geometry' column
df_stations = pd.read_csv(data_path+"\\"+"KNMI_weather_stations.csv")
geometry = [Point(lon, lat) for lon, lat in zip(df_stations['lon'], df_stations['lat'])]
gdf_points = gpd.GeoDataFrame(df_stations, geometry=geometry, crs='EPSG:4326')
gdf_points.to_file(data_path+"\\shp\\"+"station_points.shp")
# Load a world map dataset and filter the Netherlands
netherlands_shapefile1 = gpd.read_file(data_path+'\\shp\\NET_admin_0.shp')
netherlands_shapefile2 = gpd.read_file(data_path+'\\shp\\NET_admin_1.shp')
netherlands_shapefile3 = gpd.read_file(data_path+'\\shp\\agriculture_areas2.shp')
# Create a figure and axis for the plot
fig, ax = plt.subplots(figsize=(10, 10))
netherlands_shapefile2.plot(ax=ax,color='none', edgecolor='lightgray')
netherlands_shapefile1.plot(ax=ax, color='none',edgecolor='black')
# Plot the points on top of the Netherlands shapefile
gdf_points.plot(ax=ax, marker='^', color='red', markersize=30, alpha=0.5)
for x, y, label in zip(gdf_points.centroid.x, gdf_points.centroid.y, gdf_points['ID']):
    ax.text(x, y, label, fontsize=10, ha='center', va='center')
# Set the title and axis labels
ax.set_title('KNMI weather stations'+" ("+str(len(df_stations))+" stations)")
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
# Set the axis limits to focus on the Netherlands
ax.set_xlim([3, 8])
ax.set_ylim([50.5, 54])

#### PART IV - Analyzing climate data for the climatology year (i.e., 30 years)

In [ ]:
# Please provide your inputs
station_id = "ID_040"
param = "Rainfall"
season_start=1
season_end=12

In [ ]:
with open(ext_climate_pkl, 'rb') as file:
    df_f = pickle.load(file)
filtered_df = df_f[df_f.index.month.isin([season_start,season_end])]
# Sum the 'Value' column for all years
sum_by_year = filtered_df[station_id].groupby(filtered_df.index.year).sum()
# Display the sum for each year
print(sum_by_year)
# Create a time series plot
plt.figure(figsize=(10, 6))
plt.plot(sum_by_year.index, sum_by_year.values, marker='o', linestyle='-')
plt.title(param+' over the location: '+station_id+' for the months of '+str(season_start)+' and '+str(season_end))
plt.xlabel('Year')
plt.ylabel(param)
plt.grid(True)
# Fit a trend line (linear regression) to the data
slope, intercept, r_value, p_value, std_err = stats.linregress(sum_by_year.index, sum_by_year.values)
# Print a message based on the slope value
if slope < 0:
    print("Rainfall over the period shows a Decreasing Trend.")
else:
    print("Rainfall over the period shows an Increasing Trend.")
trendline = slope * np.array(sum_by_year.index) + intercept
plt.plot(sum_by_year.index, trendline, color='red', linestyle='--', label='Trend Line')
# Display the plot with the trend line
plt.grid(True)
plt.legend()
plt.show()


#### PART V - Analyzing monthly rainfall data for a specific year 

In [ ]:
# Please provide your inputs
station_id = "ID_064"
param = "Rainfall"
year = 2014

In [ ]:
# Replace '2023' with the desired year
with open(ext_climate_pkl, 'rb') as file:
    df_f = pickle.load(file)
# Sum the 'Value' column for all years
# Filter the DataFrame for the selected year
result_for_selected_year = df_f[df_f.index.year == year]
# Calculate the sum of the 'value' column for each month in the selected year
monthly_sum_for_selected_year = result_for_selected_year[station_id].groupby(result_for_selected_year.index.month).sum()
print(monthly_sum_for_selected_year)
# Define the month labels
month_labels = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
# Plot the bar chart
plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
monthly_sum_for_selected_year.plot(kind='bar')
plt.xticks(range(12), month_labels)  # Replace the default month labels with shortened labels
plt.xlabel("Month")
plt.ylabel("Total Rainfall")
# Calculate x-coordinates for the line plot
x = np.arange(len(monthly_sum_for_selected_year))
# Add a line connecting the tops of the bars
plt.plot(x, monthly_sum_for_selected_year.values, marker='^', linestyle='--', color='red')
plt.title(f"Monthly Sum for {year} for the station ID: "+station_id)
plt.show()